In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras

In [2]:
from tensorflow.keras.preprocessing.image import  load_img

In [3]:
path = './clothing-dataset-small/train/t-shirt'
name = '5f0a3fa0-6a3d-4b68-b213-72766a643de7.jpg'
fullname = f'{path}/{name}'

In [4]:
img = load_img(fullname, target_size =(299,299))

In [5]:
print(img)

<PIL.Image.Image image mode=RGB size=299x299 at 0x7FBD041A7160>


In [6]:
x = np.array(img)

# Pre-trained convolutional neural networks

In [7]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions

In [8]:
model = Xception(weights = 'imagenet', input_shape = (299,299,3))

2022-10-27 17:19:29.473549: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-10-27 17:19:29.473930: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-27 17:19:29.479724: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [9]:
X = np.array([x])
X_test = preprocess_input(X)

In [10]:
pred_test = model.predict(X_test)

2022-10-27 17:19:30.950238: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-10-27 17:19:30.950963: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1799995000 Hz


In [11]:
decode_predictions(pred_test)

[[('n03595614', 'jersey', 0.6819639),
  ('n02916936', 'bulletproof_vest', 0.038139865),
  ('n04370456', 'sweatshirt', 0.034324702),
  ('n03710637', 'maillot', 0.011354212),
  ('n04525038', 'velvet', 0.0018453546)]]

# Transfer learning

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
train_gen = ImageDataGenerator(preprocessing_function= preprocess_input)

train_dataset = train_gen.flow_from_directory('./clothing-dataset-small/train',
                              target_size=(150,150),
                              batch_size=32)

val_gen = ImageDataGenerator(preprocessing_function= preprocess_input)

val_dataset = val_gen.flow_from_directory('./clothing-dataset-small/train',
                              target_size=(150,150),
                              batch_size=32)

Found 3068 images belonging to 10 classes.
Found 3068 images belonging to 10 classes.


In [14]:
base_model = Xception(weights = 'imagenet',
                      include_top = False,
                      input_shape = (150,150,3))

base_model.trainable = False
########################################################
inputs = keras.Input(shape=(150,150,3))
base = base_model(inputs, training = False)
#polling from 3D to 2D
vectors = keras.layers.GlobalAveragePooling2D()(base)
#adding dense layer
outputs = keras.layers.Dense(10)(vectors)
model = keras.Model(inputs,outputs)
######################################################
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate = learning_rate)
loss = keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer = optimizer,
             loss = loss,
             metrics = ['accuracy'])

In [15]:
history = model.fit(train_dataset, epochs=5 , validation_data= val_dataset)

Epoch 1/5
96/96 [==============================] - 285s 3s/step - loss: 1.7357 - accuracy: 0.5740 - val_loss: 0.5321 - val_accuracy: 0.8259
Epoch 2/5
96/96 [==============================] - 268s 3s/step - loss: 0.5223 - accuracy: 0.8252 - val_loss: 0.4462 - val_accuracy: 0.8465
Epoch 3/5
96/96 [==============================] - 267s 3s/step - loss: 0.3308 - accuracy: 0.8871 - val_loss: 0.1615 - val_accuracy: 0.9449
Epoch 4/5
96/96 [==============================] - 299s 3s/step - loss: 0.2053 - accuracy: 0.9278 - val_loss: 0.1351 - val_accuracy: 0.9547
Epoch 5/5
96/96 [==============================] - 249s 3s/step - loss: 0.1405 - accuracy: 0.9488 - val_loss: 0.1237 - val_accuracy: 0.9534


# Playing with the learning rate

In [ ]:
def make_learning_model(learning_rate = 0.01):
    base_model = Xception(weights = 'imagenet',
                      include_top = False,
                      input_shape = (150,150,3))

    base_model.trainable = False
    ########################################################
    inputs = keras.Input(shape=(150,150,3))
    base = base_model(inputs, training = False)
    #polling from 3D to 2D
    vectors = keras.layers.GlobalAveragePooling2D()(base)
    #adding dense layer
    outputs = keras.layers.Dense(10)(vectors)
    model = keras.Model(inputs,outputs)
    ######################################################
    learning_rate = 0.01
    optimizer = keras.optimizers.Adam(learning_rate = learning_rate)

    loss = keras.losses.CategoricalCrossentropy(from_logits=True)
    model.compile(optimizer = optimizer,
             loss = loss,
             metrics = ['accuracy'])
    
    return model

In [ ]:
scores = {}

for lr in [0.0001,0.001,0.01,0.1]:
    
    model = make_learning_model(lr)
    history = model.fit(train_dataset, epochs=5 , validation_data= val_dataset)
    scores[lr]= history.history

In [ ]:
for lr,hist in scores.item():
    plt.plot(hist['val_accuracy'], label = 'val_acc')
    
plt.xticks(np.arange(10))
plt.legend()

In [58]:
preds = model.predict(X)

In [59]:
preds.shape

(32, 10)